![Neptune + MosaicML Composer](https://neptune.ai/wp-content/uploads/2023/09/mosiacml.svg)

# Neptune + MosaicML Composer

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/mosaicml-composer/notebooks/Neptune_MosaicML_Composer.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/mosaicml-composer/notebooks/Neptune_MosaicML_Composer.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>

<a target="_blank" href="https://app.neptune.ai/showcase/mosaicml-composer/e/MMLCOMP-3/metadata">
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>


<a target="_blank" href="https://docs.neptune.ai/integrations/mosaicml-composer/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

[MosaicML Composer](https://github.com/mosaicml/composer) is a PyTorch library for efficient neural network training.

This guide will show you how to:

* Create a Neptune logger for MosaicML Composer
* Automatically log your Composer training metadata to Neptune

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U neptune torch torchvision #mosaicml

# TODO: Update after released
%pip install git+https://github.com/AleksanderWWW/composer.git@neptune-logger

## Import libraries

In [ ]:
import torch

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from composer import Trainer
from composer.models import mnist_model
from composer.loggers import NeptuneLogger
from composer.algorithms import LabelSmoothing, ProgressiveResizing
from composer.callbacks import ImageVisualizer

## Prepare dataset and dataloaders

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST("data", download=True, train=True, transform=transform)
eval_dataset = datasets.MNIST("data", download=True, train=False, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32)
eval_dataloader = DataLoader(eval_dataset, batch_size=16)

## Create `neptune_logger`

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/mosaicml-composer](https://app.neptune.ai/o/common/org/mosaicml-composer). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

neptune_logger = NeptuneLogger(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["mnist", "notebook"],  # (optional) use your own
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app.

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
from neptune import ANONYMOUS_API_TOKEN

neptune_logger = NeptuneLogger(
    api_token=ANONYMOUS_API_TOKEN,  # Replace with your own
    project="common/mosaicml-composer",  # Replace with your own
    capture_hardware_metrics=True,  # This is turned off by default in Notebooks
    tags=["mnist", "notebook"],  # (optional) use your own
)

## Configure Composer algorithms

In [ ]:
label_smoothing = LabelSmoothing(0.1)

In [ ]:
prog_resize = ProgressiveResizing(
    initial_scale=0.6,
    finetune_fraction=0.34,
)

## Train model with Composer algorithms

In [ ]:
trainer = Trainer(
    model=mnist_model(),
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    max_duration="3ep",
    device="gpu" if torch.cuda.is_available() else "cpu",
    algorithms=[label_smoothing, prog_resize],
    callbacks=ImageVisualizer(),  # Adding the ImageVisualizer() callback automatically logs input images to Neptune
    loggers=neptune_logger,
)

trainer.fit()

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

## Log additional metadata
`neptune_logger`'s `base_handler` property exposes the [Namespace handler](https://docs.neptune.ai/api/field_types/#namespace-handler) being used by the logger. You can use it to log any additional metadata in the **base namespace used by the logger**.

Read more about namespaces [in the docs](https://docs.neptune.ai/about/namespaces_and_fields/).

**Note:** The default base namespace used by `NeptuneLogger` is "training". You can update this by passing your own "base_namespace" while initializing `NeptuneLogger`.

In the below example, we log a sample image from the training dataset to the "training/sample_image" namespace of the run.

In [ ]:
from neptune.types import File

neptune_logger.base_handler["sample_image"].upload(File.as_image(train_dataset.data[0] / 255))

## Log to your custom namespace
If you want to log to a different base namespace than used by the base handler, you can use `neptune_logger`'s `neptune_run` property to access the underlying Neptune `Run` object and pass your own namespaces.

In the below example, we log a sample image from the eval dataset to the "eval/sample_image" namespace of the run.

In [ ]:
neptune_logger.neptune_run["eval/sample_image"].upload(File.as_image(eval_dataset.data[0] / 255))

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
trainer.close()

## Analyze run in the Neptune app
Follow the run link in the above cell output and explore the logged metadata.  
You can also explore this [example run](https://app.neptune.ai/showcase/mosaicml-composer/e/MMLCOMP-3/metadata).